# Example of the ``Splitter`` class usage for solving group splitting problem

In this tutorial we will use *Amrosia* splitting tools to create a number of groups using different strategies.

Group splitting problem usually appears in A/B testing when we have designed experiment parameters and want to create experimental groups consist from the objects of the research.

## Two different splitting paradigms

Basically, the splitting of objects into groups is divided into *batch* and *real-time split* approaches. 

For the first type of splitting we precalculate the contents of our experimental groups using, for example, a common database with research objects. \
In the second type of splitting approach some tools distribute objects into groups in real time as they arrive, although it may also use some pre-calculated information.

Further in this tutorial we will review the tools for batch splitting tasks.

**Note:** *Ambrosia* now supports only batch spliiting. Real-time splitting tools are under development.

## Let's start the tutorial

In [1]:
import sys, os
sys.path.insert(1, os.path.realpath(os.path.pardir))

In [7]:
import pandas as pd
import numpy as np

import yaml

from ambrosia.splitter import Splitter, split, load_from_config

Generate synthetic data with a number of defferent columns.\
We will create 200000 objects with unique id and some numerical features

In [21]:
np.random.seed(42)

dataframe = pd.DataFrame({
    'm': np.zeros((200000, )),
    'a': np.random.normal(size=200000),
    'b': np.random.normal(size=200000)
})
dataframe['l'] = np.where(df2['a'] > 0, 1, 0)
dataframe['e'] = np.where(df2['b'] > 0, 1, 0)
dataframe['object_id'] = np.random.choice(dataframe.index,
                                          size=df2.shape[0],
                                          replace=False)
dataframe.head()

,m,a,b,l,e,object_id
0,0.0,0.496714,1.561841,0,1,63869
1,0.0,-0.138264,-0.094228,1,1,82374
2,0.0,0.647689,-1.329536,1,0,162918
3,0.0,1.523030,-1.388638,1,0,36327
4,0.0,-0.234153,-0.342651,0,1,91526


In [22]:
dataframe.shape

(200000, 6)

Now let's get acquainted with the ``Splitter`` class.

The ``Splitter`` class is *Ambrosia's* main tool for splitting objects into the creating groups. It has one main public method ``run()`` which returns the table with a groups of the desired size.

Let's create an instance of the class and pass to the constructor generated data ``dataframe`` about objects *(this data is like some abstract user database)* which will be used further for the creation of the groups using different methods. We also specify for ``id_column``  a column ``"object_id"``  that contains unique identifiers of objects. If this column had not been specified, dataframe indexes will be used as identifiers.

In [23]:
splitter = Splitter(dataframe=dataframe, id_column='object_id')

As well as in the ``Designer`` class, we can pass this dataframe and other parameters later as an argument to the ``run()`` method. We can do the same with most of the parameters related directly to the experiment (errors, effects, and so on) - either pass them to the constructor during initialization (and then they will become attributes of the created instance), or pass them later, when execute ``run()`` method. In case of parameter selection ambiguity, the argument in the method takes precedence over the attribute value.

Now let's move on to review different ways to create groups that are implemented in the ``Splitter`` class.

## Split approaches

### Simple split

The first type of splitting strategy is called ``"simple"`` and is really about a very simple, non-deterministic way of creating groups, in which a new result is produced each time it is executed.

To create such split we need to execute ``run()`` method with corresponding value of ``method`` parameter. We will create groups each of size 2000 objects.

In [31]:
splitter.run(method='simple', groups_size=2000)

,m,a,b,l,e,object_id,group
169442,0.0,-1.264081,-0.843481,0,1,197575,A
186911,0.0,-0.146136,1.708641,0,0,98109,A
194145,0.0,-1.411002,0.323674,1,1,34031,A
33386,0.0,0.201264,1.322248,0,0,199765,A
129400,0.0,-0.261651,0.043902,0,1,112889,A
...,...,...,...,...,...,...,...
49247,0.0,-0.039739,-0.649425,1,1,57936,B
63700,0.0,-0.961996,1.400426,1,1,122455,B
93594,0.0,0.486525,-0.749344,1,1,47744,B
66644,0.0,0.600458,0.042417,0,0,104374,B


### Hash split 

The hash split strategy is based on hashing object identifiers and distributing the resulting hash values into appropriate groups. \
This method allows you to perform a deterministic split of objects into groups, also there is no need for a tables with the assigned group labels, because this splitting method allows  to restore the labels at any time by re-execution. 

To make the splits for each experiment unique, the ``"salt"`` parameter is used, which is appended to the end of the identifier of each object. The salt value can be, for example, the name of the experiment being performed.

You can read more about hash-based splitting on the web.

Let's create a hash split and make sure the result is deterministic

In [33]:
groups_size= 5000
salt = 'example_dummy_experiment_2023'

Execute split with pre-defined salt value

In [34]:
splitter.run(method='hash', groups_size=groups_size, salt=salt)

,m,a,b,l,e,object_id,group
14,0.0,-1.724918,-0.350186,0,0,90837,A
44,0.0,-1.478522,0.166608,0,0,123196,A
64,0.0,0.812526,0.914659,1,1,117133,A
65,0.0,1.356240,0.731410,0,1,144787,A
161,0.0,0.787085,-1.012367,1,0,186437,A
...,...,...,...,...,...,...,...
199760,0.0,0.172396,0.844596,0,0,166816,B
199783,0.0,-0.477993,-0.899310,0,0,134168,B
199867,0.0,-1.164759,-0.649031,1,0,41423,B
199868,0.0,0.162848,2.835048,0,0,33513,B


Then get a similar groups for the same salt value

In [35]:
splitter.run(method='hash', groups_size=groups_size, salt=salt)

,m,a,b,l,e,object_id,group
14,0.0,-1.724918,-0.350186,0,0,90837,A
44,0.0,-1.478522,0.166608,0,0,123196,A
64,0.0,0.812526,0.914659,1,1,117133,A
65,0.0,1.356240,0.731410,0,1,144787,A
161,0.0,0.787085,-1.012367,1,0,186437,A
...,...,...,...,...,...,...,...
199760,0.0,0.172396,0.844596,0,0,166816,B
199783,0.0,-0.477993,-0.899310,0,0,134168,B
199867,0.0,-1.164759,-0.649031,1,0,41423,B
199868,0.0,0.162848,2.835048,0,0,33513,B


Split result will be different if the salt is changed

In [37]:
splitter.run(method='hash', groups_size=groups_size, salt='salt')

,m,a,b,l,e,object_id,group
43,0.0,-0.301104,0.440295,1,0,139147,A
192,0.0,0.214094,0.021427,1,0,231,A
226,0.0,0.064280,1.553626,1,0,139761,A
235,0.0,0.633919,-1.277988,0,1,153281,A
285,0.0,-1.952088,1.610653,1,0,36040,A
...,...,...,...,...,...,...,...
199862,0.0,2.035899,0.452816,0,0,34064,B
199949,0.0,0.438721,-0.592572,1,1,99013,B
199970,0.0,0.868163,0.463027,0,1,53783,B
199991,0.0,0.383196,0.230814,0,1,199822,B


If no salt argument is passed, a random value will be generated during the split.

**Hash splitting method is fast and convenient and is recommended to use by default.**

### Metric split

For some tasks, it is very useful to find similar objects and distribute them into groups. For example, we can choose a random object in group A and from the general pool find the closest neighbor to it by some metric and send it to group B. This will make the groups more similar and increase the power of some statistical tests, which is especially valuable for small groups.

This approach is implemented in the ``"metric"`` split method, we can specify a set of features using ``fit_columns`` parameter, based on which pairs of similar objects will be selected using minimization of the Euclidean distance and distributed between the groups.

We will create two groups using metric split based on two features ``a`` and ``b``. Metric split requires sufficient computational resources to find nearest neighbors to set of points equal to size of one group.

In [41]:
metric_split = splitter.run(method='metric', groups_size=groups_size, fit_columns=['a', 'b'])

In [42]:
metric_split

,m,a,b,l,e,object_id,group
22850,0.0,0.969768,-0.705291,1,0,124320,A
144144,0.0,0.466991,-0.967875,0,1,173341,A
23853,0.0,0.780214,0.894208,0,0,63407,A
129993,0.0,-0.012896,-0.953599,1,1,148856,A
126999,0.0,0.262705,-0.126261,0,0,78810,A
...,...,...,...,...,...,...,...
149267,0.0,-1.330439,0.390575,1,0,75453,B
151571,0.0,-0.356644,0.794770,1,0,172434,B
61357,0.0,-0.026874,0.806196,0,1,172192,B
152968,0.0,1.023680,0.231463,0,1,194383,B


Currently, pairs of similar objects occupy the same positions in group slices, and that is the only way to find them if you want to inspect individually.

In [47]:
metric_split.query("group == 'A'").iloc[0]

m                 0.0
a            0.969768
b           -0.705291
l                   1
e                   0
object_id      124320
group               A
Name: 22850, dtype: object

In [48]:
metric_split.query("group == 'B'").iloc[0]

m                 0.0
a            0.967779
b           -0.707851
l                   1
e                   0
object_id       96793
group               B
Name: 24997, dtype: object

**Note:** Metric split creates pairs (or sets in the case of multiple groups) of dependent objects between groups. This leads to the need to **use paired statistical tests**.

## Stratification

We can sample groups based with stratification. 

The stratification technique makes groups more homogeneous and similar to the general population from which these groups were sampled, as well as to reduce the dispersion of metrics in groups. This may be especially usefull in the case of small groups.

To demonstrate let's choose a binary column for stratification and pass it to ``strat_columns`` parameter, and see the ratios of the feature distribution in the case of stratification and without it

In [88]:
groups_size = 500

In [89]:
stratified_split = splitter.run(method='simple', groups_size=groups_size, strat_columns=['l'])
non_stratified_split = splitter.run(method='simple', groups_size=groups_size)

In [90]:
print(f'Initial share of strata: {dataframe["l"].mean() * 100:.1f}%')

Initial share of strata: 50.0%


Share of strata inside the splits

In [99]:
print(
    f'Share of strata in groups with stratification: {stratified_split["l"].mean() * 100:.1f}%'
)
print(
    f'Share of strata in groups without stratification: {non_stratified_split["l"].mean() * 100:.1f}%'
)

Share of strata in groups with stratification: 49.9%
Share of strata in groups without stratification: 51.2%


Share of strata inside the groups

In [100]:
print('Share of strata in each group with stratification\n',
      np.round(stratified_split.groupby('group')['l'].mean(), 3))
print('\n\nShare of strata in each group without stratification\n',
      np.round(non_stratified_split.groupby('group')['l'].mean(), 3))

Share of strata in each group with stratification
 group
A    0.498
B    0.500
Name: l, dtype: float64


Share of strata in each group without stratification
 group
A    0.510
B    0.514
Name: l, dtype: float64


## Multigroup split

Often, two experimental groups are not enough, for example, when we want to test the performance of multiple new recommender system algorithms. For that scenario one may want to make A/B/C/.. split.

In *Ambrosia*, all  functions and methods above are generalized for split into several groups and the number of groups can be controlled using ``groups_number`` parameter.

Let's create 3 groups using metric split

In [49]:
metric_multisplit = splitter.run(method='metric',
                                 groups_size=groups_size,
                                 fit_columns=['a', 'b'],
                                 groups_number=3)

In [52]:
metric_multisplit.query("group == 'A'").iloc[0]

m                 0.0
a            0.207803
b            0.446489
l                   0
e                   0
object_id      141744
group               A
Name: 3417, dtype: object

In [53]:
metric_multisplit.query("group == 'B'").iloc[0]

m                 0.0
a            0.203121
b            0.442142
l                   1
e                   0
object_id      115623
group               B
Name: 162660, dtype: object

In [54]:
metric_multisplit.query("group == 'C'").iloc[0]

m                 0.0
a            0.206613
b            0.443756
l                   0
e                   1
object_id       85901
group               C
Name: 199756, dtype: object

And now create 10 groups using hash method

In [56]:
hash_multisplit = splitter.run(method='hash',
                               groups_size=1000,
                               groups_number=10)

In [57]:
hash_multisplit

,m,a,b,l,e,object_id,group
36,0.0,0.208864,0.524205,1,0,172915,A
208,0.0,0.515048,-0.678438,0,0,176441,A
250,0.0,-1.260884,2.688230,1,1,161465,A
443,0.0,-0.089120,-1.183396,0,0,87055,A
458,0.0,-0.553649,-0.360052,1,1,79178,A
...,...,...,...,...,...,...,...
199249,0.0,1.255824,-0.311677,0,1,18141,J
199317,0.0,0.565089,-0.397288,0,0,125400,J
199569,0.0,1.778133,-0.960608,1,1,121432,J
199702,0.0,-2.924105,1.870715,0,0,171345,J


In [58]:
hash_multisplit.group.value_counts()

A    1000
B    1000
C    1000
D    1000
E    1000
F    1000
G    1000
H    1000
I    1000
J    1000
Name: group, dtype: int64

## Splitting the full table

Sometimes there are scenarios where one need to divide an entire table into groups. At the moment, *Ambrosia* allows to split data frames into 2 groups using the ``part_of_table``.

We will split passed data frame in a ratio of 1/3 (group A to B) using hash method

In [80]:
part_of_table = 1/3
fractional_hash_split = splitter.run(method='hash',
                                     part_of_table=part_of_table,
                                     salt='fractional_split')

In [81]:
fractional_hash_split

,m,a,b,l,e,object_id,group
1,0.0,-0.138264,-0.094228,1,1,82374,A
3,0.0,1.523030,-1.388638,1,0,36327,A
5,0.0,-0.234137,-1.580520,1,0,63304,A
6,0.0,1.579213,0.587148,0,0,187546,A
15,0.0,-0.562288,-1.362157,1,0,133839,A
...,...,...,...,...,...,...,...
199991,0.0,0.383196,0.230814,0,1,199822,B
199994,0.0,-0.590488,-0.518154,1,0,12123,B
199996,0.0,0.565654,-2.316381,1,1,147356,B
199998,0.0,0.855673,0.462531,1,1,132270,B


In [82]:
fractional_hash_split.group.value_counts(normalize=True)

B    0.666665
A    0.333335
Name: group, dtype: float64

## Selection of an existing group for a test group

Another type of scenario that sometimes occurs in A/B testing tasks, is a problem of post-generation of a control group from the total available pool of objects that were not affected by the treatment.

Although you have to be quite careful in post-analysis of experiments and in post-generation of samples, *Ambrosia* allows to create control group to the existing test using all methods above.

To do this, it is enough to pass a list of identifiers from the test group to ``test_group_ids`` parameter.

In [59]:
np.random.seed(42)
group_size = 10000
test_ids = np.random.choice(dataframe.object_id, size=group_size, replace=False)

In [60]:
post_hash_split = splitter.run(method='hash',
                               groups_size=groups_size,
                               test_group_ids=test_ids,
                               salt='post-split')

In [71]:
post_hash_split

,m,a,b,l,e,object_id,group
24,0.0,-0.544383,0.242347,0,0,37916,A
94,0.0,-0.392108,0.026810,1,0,12345,A
136,0.0,-0.783253,-1.911507,0,0,94132,A
152,0.0,-0.680025,-0.649503,1,0,87931,A
155,0.0,-0.714351,0.266708,1,1,122498,A
...,...,...,...,...,...,...,...
199945,0.0,2.866851,-0.048362,1,0,146867,B
199963,0.0,-0.935605,0.169802,1,1,116839,B
199976,0.0,-0.811208,-0.931313,0,0,18652,B
199977,0.0,0.547009,0.330221,0,0,180593,B


Check that all objects with test ids are in group B and not in A

In [69]:
np.isin(test_ids, post_hash_split.query("group == 'A'").object_id).sum()

0

In [70]:
np.isin(test_ids, post_hash_split.query("group == 'B'").object_id).sum()

10000

---

## Learn more

There is some more information about groups split using *Ambrosia*

Check:
- An example of splitting groups from a Spark DataFrame (currently has a slightly reduced functionality)